<a href="https://colab.research.google.com/github/Stealth-Food-Exploration/web-application/blob/main/Research/ML/IndianFoodDetection_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os.path
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D,Flatten,Dropout,GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive


In [ ]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [ ]:
from pathlib import Path


In [ ]:
image_dir = Path('/content/drive/My Drive/Indian Food Images')

In [ ]:
print(image_dir)

/content/drive/My Drive/Indian Food Images


In [ ]:
if image_dir.exists():
    print("Directory exists")
else:
    print("Directory does not exist")

Directory exists


In [ ]:
# image_dir' is a Path object representing the directory containing images
# and we get a list of all JPG files within it recursively
filepath = list(image_dir.glob(r'**/*.jpg'))

# Extracting labels from file paths by taking the parent directory name of each file
label = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))
# Converting the list of file paths to a pandas Series and setting its name as 'Filepath'
filepath = pd.Series(filepath, name='Filepath').astype(str)

# Creating a pandas Series for labels with the name 'Label'
label = pd.Series(label, name='Label')

# Concatenating the 'filepath' and 'label' Series along the columns axis
# Sampling the concatenated DataFrame to shuffle the rows (frac=1.0 means keep all rows, random_state=1 for reproducibility)
# Resetting the index of the DataFrame to reindex the shuffled rows
image_df = pd.concat([filepath, label], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

# Displaying the first few rows of the DataFrame
image_df.head()

,Filepath,Label
0,/content/drive/My Drive/Indian Food Images/ana...,anarsa
1,/content/drive/My Drive/Indian Food Images/lya...,lyangcha
2,/content/drive/My Drive/Indian Food Images/pal...,palak_paneer
3,/content/drive/My Drive/Indian Food Images/pan...,paneer_butter_masala
4,/content/drive/My Drive/Indian Food Images/but...,butter_chicken


In [ ]:
# splitting dataset into training and testing data
train_df, test_df = train_test_split(image_df, test_size = 0.20,random_state = 43)

In [ ]:
train_df['Label']

593              basundi
338                 naan
2448       chicken_tikka
2274             bhatura
34        gajar_ka_halwa
              ...       
2325             gavvalu
3889    double_ka_meetha
2303        pootharekulu
3392        palak_paneer
1860           adhirasam
Name: Label, Length: 3200, dtype: object

In [ ]:
# ImageDataGenerator for training data
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    # Rescale pixel values to [0,1] range
    rescale=1./255,
    # Fraction of images reserved for validation (20%)
    validation_split=0.2,
    # Range for random rotations (degrees)
    rotation_range=20,
    # Range for random horizontal shifts
    width_shift_range=0.2,
    # Range for random vertical shifts
    height_shift_range=0.2,
    # Shear intensity (shear angle in counter-clockwise direction in degrees)
    shear_range=0.2,
    # Range for random zoom
    zoom_range=0.2,
    # Randomly flip inputs horizontally
    horizontal_flip=True,
    # Strategy for filling in newly created pixels after a rotation or a width/height shift
    fill_mode='nearest'
)

# ImageDataGenerator for test data (only rescaling)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = train_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    color_mode='rgb',  # RGB color mode
    class_mode='categorical',  # Use categorical labels
    shuffle=True,
    seed=42,
    subset='training'  # Subset for training
)

validation_dataset = train_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=42,
    subset='validation'  # Subset for validation
)

test_dataset = test_gen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False  # No shuffling for test dataset
)

Found 2560 validated image filenames belonging to 80 classes.
Found 640 validated image filenames belonging to 80 classes.
Found 800 validated image filenames belonging to 80 classes.


In [ ]:
# Assuming 'train_gen' is your ImageDataGenerator and 'train_dataset' is your DataFrameIterator
class_indices = train_dataset.class_indices

# Print the class indices dictionary
print("Class Indices:")
print(class_indices)

Class Indices:
{'adhirasam': 0, 'aloo_gobi': 1, 'aloo_matar': 2, 'aloo_methi': 3, 'aloo_shimla_mirch': 4, 'aloo_tikki': 5, 'anarsa': 6, 'ariselu': 7, 'bandar_laddu': 8, 'basundi': 9, 'bhatura': 10, 'bhindi_masala': 11, 'biryani': 12, 'boondi': 13, 'butter_chicken': 14, 'chak_hao_kheer': 15, 'cham_cham': 16, 'chana_masala': 17, 'chapati': 18, 'chhena_kheeri': 19, 'chicken_razala': 20, 'chicken_tikka': 21, 'chicken_tikka_masala': 22, 'chikki': 23, 'daal_baati_churma': 24, 'daal_puri': 25, 'dal_makhani': 26, 'dal_tadka': 27, 'dharwad_pedha': 28, 'doodhpak': 29, 'double_ka_meetha': 30, 'dum_aloo': 31, 'gajar_ka_halwa': 32, 'gavvalu': 33, 'ghevar': 34, 'gulab_jamun': 35, 'imarti': 36, 'jalebi': 37, 'kachori': 38, 'kadai_paneer': 39, 'kadhi_pakoda': 40, 'kajjikaya': 41, 'kakinada_khaja': 42, 'kalakand': 43, 'karela_bharta': 44, 'kofta': 45, 'kuzhi_paniyaram': 46, 'lassi': 47, 'ledikeni': 48, 'litti_chokha': 49, 'lyangcha': 50, 'maach_jhol': 51, 'makki_di_roti_sarson_da_saag': 52, 'malapua': 

In [ ]:
index_to_class = {v: k for k, v in class_indices.items()}
print("Index to Class Mapping:")
print(index_to_class)

Index to Class Mapping:
{0: 'adhirasam', 1: 'aloo_gobi', 2: 'aloo_matar', 3: 'aloo_methi', 4: 'aloo_shimla_mirch', 5: 'aloo_tikki', 6: 'anarsa', 7: 'ariselu', 8: 'bandar_laddu', 9: 'basundi', 10: 'bhatura', 11: 'bhindi_masala', 12: 'biryani', 13: 'boondi', 14: 'butter_chicken', 15: 'chak_hao_kheer', 16: 'cham_cham', 17: 'chana_masala', 18: 'chapati', 19: 'chhena_kheeri', 20: 'chicken_razala', 21: 'chicken_tikka', 22: 'chicken_tikka_masala', 23: 'chikki', 24: 'daal_baati_churma', 25: 'daal_puri', 26: 'dal_makhani', 27: 'dal_tadka', 28: 'dharwad_pedha', 29: 'doodhpak', 30: 'double_ka_meetha', 31: 'dum_aloo', 32: 'gajar_ka_halwa', 33: 'gavvalu', 34: 'ghevar', 35: 'gulab_jamun', 36: 'imarti', 37: 'jalebi', 38: 'kachori', 39: 'kadai_paneer', 40: 'kadhi_pakoda', 41: 'kajjikaya', 42: 'kakinada_khaja', 43: 'kalakand', 44: 'karela_bharta', 45: 'kofta', 46: 'kuzhi_paniyaram', 47: 'lassi', 48: 'ledikeni', 49: 'litti_chokha', 50: 'lyangcha', 51: 'maach_jhol', 52: 'makki_di_roti_sarson_da_saag', 53

In [ ]:
import pickle

pickle.dump(index_to_class,open('/content/drive/My Drive/index_to_class.pkl','wb'))

In [ ]:
import json
with open('/content/drive/My Drive/index_to_class.json', 'w') as f:
    json.dump(index_to_class, f)

# Build the CNN

Using Mobile Net V2 we are training the Imagenet to use our Indian food Images

This model is overfitting. Since the training accuracy is significantly higher than the validation accuracy and the validation accuracy has plateaued or even slightly decreased while the training accuracy remains high, it suggests that the model may be overfitting on the training data and not generalizing well to new, unseen data. It could benefit from strategies to combat overfitting, such as adding dropout, using more training data, or applying regularization techniques.

In [ ]:
# Load the pre-trained VGG19 model (excluding the top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create your custom model on top of the base model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Use the pre-trained base without training
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # Add more units based on your requirements
x = Dense(256, activation='relu')(x)  # Add more units based on your requirements
x = Dense(128, activation='relu')(x)
outputs = Dense(80, activation='softmax')(x)  # Use softmax for multi-class classification

# Create the final model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_history = model.fit(train_dataset, validation_data = validation_dataset, epochs = 30)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/30
15/80 [====>.........................] - ETA: 11:17 - loss: 4.4154 - accuracy: 0.0250

KeyboardInterrupt: 

In [ ]:
# saving model in the kaggle/working dir
model.save('/content/drive/My Drive/IndianFood_model_v1')

NameError: name 'model' is not defined

In [ ]:
# used to convert the saved model directory into a zip file so that we can download it

import shutil

# Replace 'source_directory' with the path to your directory
source_directory = '/content/drive/My Drive/IndianFood_model_v1'
shutil.make_archive('/content/drive/My Drive/IndianFood_model_v1', 'zip', source_directory)

'/content/drive/My Drive/IndianFood_model_v1.zip'

In [ ]:
import requests

url = 'http://localhost:5000/predict'  # Change to your Flask app's URL
image_path = '/content/drive/My Drive/test_1.jpg'

with open(image_path, 'rb') as f:
    files = {'image': f}
    response = requests.post(url, files=files)

print(response.json())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/test_1.jpg'